# Fragmentation Gain Intensity Analysis per Municipalities
## Workflow
1. Rasterize municipalities vector with municipality id.
2. Categorize gain, loss, presence persistence, absence persistence layer for framgnetation, non- fragmentation, and non- forest.
3. Count the gain, loss, presence perisistence, absence persistence intensity for each municipalities.
    - Group the row values by municipality id.
    - Count the interest, forest at 2006, forest at 2016.
    - Calculate the intensity.
4. Join the result data frame into the original municipal df.
5. Calculate local moran's i & Gi* statistics.

In [1]:
import rasterio as rio 
import geopandas as gpd 
import numpy as np 
import pandas as pd

In [2]:
import os
os.chdir("/Volumes/volume 1/GIS Projects/Research/byTown")

## Step1

### Add Municipality ID

In [3]:
# READ DATA####
v = gpd.read_file("./townssurvey_shp/TOWNSSURVEY_POLY.shp")
# ADD ID COLUMN####
v["townID"] = range(1, len(v) + 1)
# EXPORT THE DATA####
# v.to_file("town_with_id.shp")

### Rasterize Polygon with ID

In [4]:
def rasterizeVctWVal(vctPath: str,
                     rstPath: str,
                     valColumn: str,
                     outPath: str):
    v = gpd.read_file(vctPath)
    r = rio.open(rstPath)

    geomval = ((geom, val) for geom, val in zip(v.geometry, v[valColumn]))

    from rasterio.features import rasterize
    outr = rasterize(geomval,
                     out_shape = r.shape,
                     transform = r.transform,
                     all_touched = True,
                     fill = 0,
                     dtype = np.uint16)
    
    with rio.open(
        outPath, "w",
        driver = "GTiff",
        crs = r.crs,
        transform = r.transform,
        dtype = np.uint16,
        count = 1,
        width = r.width,
        height = r.height) as newr:
        newr.write(outr, indexes = 1)


In [5]:
# rasterizeVctWVal("town_with_id.shp",
#                  "fad2006_4conn_all.tif",
#                  "townID",
#                  "towns.tif")

## Step2

### Assign values representing gain(1), loss(2), presence persistence(3), and absence persistence(4).

In [6]:
def reclassifyChanges(xPath,
                      yPath,
                      targetVal,
                      outPath,
                      outOfBoundaryVal = 0):
    
    xarr = rio.open(xPath).read(1)
    yarr = rio.open(yPath).read(1)

    outarr = xarr.copy()

    outarr[(xarr != targetVal) & (yarr == targetVal)] = 1
    outarr[(xarr == targetVal) & (yarr != targetVal)] = 2
    outarr[(xarr == targetVal) & (yarr == targetVal)] = 3
    outarr[(xarr != targetVal) & (yarr != targetVal)] = 4
    outarr[(xarr == outOfBoundaryVal) | (yarr == outOfBoundaryVal)] = 0


    ref = rio.open(xPath)
    outMeta = ref.meta.copy()
    outMeta.update({
        "dirver": "GTiff",
        "height": ref.height,
        "width": ref.width,
        "transform": ref.transform,
        "nodata": 0,
        "dtype": np.uint8,
        "crs": ref.crs
    })

    with rio.open(outPath, "w", **outMeta) as newr:
        newr.write(outarr, 1)

In [ ]:
# for i in range(1, 4):
#     reclassifyChanges("fad2006_4conn_all.tif",
#                       "fad2016_4conn_all.tif",
#                       i,
#                       f"fad_changes_{i}.tif")

## Step3

### Count Foreground, Background, and Extent Size for Each Municipality ID at each time point
Output = dfs which contains size and municipality id

In [7]:
def countPixels(forestRasterPath, 
                municipalRaster,
                valueOfInterest):
    fr_original = rio.open(forestRasterPath).read(1)
    mr_original = rio.open(municipalRaster).read(1)
    fr = fr_original[(mr_original != 0) & (fr_original != 0)].flatten()
    print(np.unique(fr))
    mr = mr_original[(mr_original != 0) & (fr_original != 0)].flatten()

    fr_for = np.zeros(np.size(fr)).astype(np.uint8)
    fr_for[(fr == valueOfInterest)] = 1
    
    print(np.unique(fr_for))

    colname1 = "fore"

    df = pd.DataFrame({
        "muni": mr,
        colname1: fr_for,
        "Extent": 1
    })

    df_out = df.groupby(by = ["muni"]).sum()
    df_out[f"bac_{valueOfInterest}"] = (df_out["Extent"] - df_out[colname1]).astype(np.uint64)
    df_out[f"fore_intensity"] = (df_out[colname1]/ df_out["Extent"])* 100

    return df_out


In [8]:
# Count Fragmented Forest at 2006====
frag2006 = countPixels("fad2006_4conn_all.tif",
                       "towns.tif",
                       1)
frag2006

[1 2 3]
[0 1]


,fore,Extent,bac_1,fore_intensity
muni,,,,
2,4,11,7,36.363636
8,0,3,3,0.000000
9,0,23,23,0.000000
10,0,40,40,0.000000
11,0,7,7,0.000000
...,...,...,...,...
1232,0,4,4,0.000000
1235,0,8,8,0.000000
1236,5179,61284,56105,8.450819


In [9]:
# Count Non Fragmented Forest at 2006====
nonfrag2006 = countPixels("fad2006_4conn_all.tif",
                          "towns.tif",
                          2)
nonfrag2006

[1 2 3]
[0 1]


,fore,Extent,bac_2,fore_intensity
muni,,,,
2,0,11,11,0.000000
8,0,3,3,0.000000
9,0,23,23,0.000000
10,0,40,40,0.000000
11,0,7,7,0.000000
...,...,...,...,...
1232,0,4,4,0.000000
1235,0,8,8,0.000000
1236,33984,61284,27300,55.453299


In [10]:
# Count Non Forest at 2006====
nonforest2006 = countPixels("fad2006_4conn_all.tif",
                            "towns.tif",
                            3)
nonforest2006

[1 2 3]
[0 1]


,fore,Extent,bac_3,fore_intensity
muni,,,,
2,7,11,4,63.636364
8,3,3,0,100.000000
9,23,23,0,100.000000
10,40,40,0,100.000000
11,7,7,0,100.000000
...,...,...,...,...
1232,4,4,0,100.000000
1235,8,8,0,100.000000
1236,22121,61284,39163,36.095881


In [11]:
# Count Classes at 2016====
frag2016 = countPixels("fad2016_4conn_all.tif",
                       "towns.tif",
                       1)
nonfrag2016 = countPixels("fad2016_4conn_all.tif",
                       "towns.tif",
                       2)
nonforest2016 = countPixels("fad2016_4conn_all.tif",
                            "towns.tif",
                            3)

[1 2 3]
[0 1]
[1 2 3]
[0 1]
[1 2 3]
[0 1]


In [12]:
frag2016

,fore,Extent,bac_1,fore_intensity
muni,,,,
2,4,11,7,36.363636
8,0,3,3,0.000000
9,0,23,23,0.000000
10,0,40,40,0.000000
11,0,7,7,0.000000
...,...,...,...,...
1232,0,4,4,0.000000
1235,0,8,8,0.000000
1236,8367,61284,52917,13.652829


In [13]:
nonfrag2016

,fore,Extent,bac_2,fore_intensity
muni,,,,
2,0,11,11,0.00000
8,0,3,3,0.00000
9,0,23,23,0.00000
10,0,40,40,0.00000
11,0,7,7,0.00000
...,...,...,...,...
1232,0,4,4,0.00000
1235,0,8,8,0.00000
1236,30247,61284,31037,49.35546


In [14]:
nonforest2016

,fore,Extent,bac_3,fore_intensity
muni,,,,
2,7,11,4,63.636364
8,3,3,0,100.000000
9,23,23,0,100.000000
10,40,40,0,100.000000
11,7,7,0,100.000000
...,...,...,...,...
1232,4,4,0,100.000000
1235,8,8,0,100.000000
1236,22670,61284,38614,36.991711


### Sum up the G, L, Pp, Ap by Each Municipality
Output = df which contains the number of pixels of each categories and municipality id.

In [21]:
def aggregateByMuni(forestRasterPath,
                    municipalRaster,
                    valueOfInterest):
    
    colName = "val"

    fr = rio.open(forestRasterPath).read(1)
    mr = rio.open(municipalRaster).read(1)
    fr2 = fr[(mr != 0) & (fr != 0)].flatten()
    mr2 = mr[(mr != 0) & (fr != 0)].flatten()

    fr_for = np.zeros(np.size(fr2)).astype(np.uint8)
    fr_for[(fr2 == valueOfInterest)] = 1

    df = pd.DataFrame({
        "muni": mr2,
        colName: fr_for
    })

    df_out = df.groupby(by = ["muni"]).sum()

    return df_out

In [22]:
# Count Pixel of Fragmented Forest Gain====
frag_gain = aggregateByMuni("fad_changes_1.tif",
                            "towns.tif",
                            1)


In [23]:
frag_gain

,val
muni,
2,0
8,0
9,0
10,0
11,0
...,...
1232,0
1235,0
1236,3344


In [24]:
frag_loss = aggregateByMuni("fad_changes_1.tif",
                            "towns.tif",
                            2)
frag_loss

,val
muni,
2,0
8,0
9,0
10,0
11,0
...,...
1232,0
1235,0
1236,156


In [25]:
frag_pp = aggregateByMuni("fad_changes_1.tif",
                          "towns.tif",
                          3)
frag_pp

,val
muni,
2,4
8,0
9,0
10,0
11,0
...,...
1232,0
1235,0
1236,5023


In [26]:
frag_ap = aggregateByMuni("fad_changes_1.tif",
                          "towns.tif",
                          4)
frag_ap

,val
muni,
2,7
8,3
9,23
10,40
11,7
...,...
1232,4
1235,8
1236,52761


In [27]:
nonfrag_gain = aggregateByMuni("fad_changes_2.tif",
                               "towns.tif",
                               1)
nonfrag_loss = aggregateByMuni("fad_changes_2.tif",
                               "towns.tif",
                               2)
nonfrag_pp = aggregateByMuni("fad_changes_2.tif",
                             "towns.tif",
                             3)
nonfrag_ap = aggregateByMuni("fad_changes_2.tif",
                             "towns.tif",
                             4)

In [28]:
nonfrag_gain["val"].max()

3898

In [29]:
frag_gain["val"].max()

5396

In [30]:
nonfrag_loss["val"].max()

9460

In [31]:
frag_loss["val"].max()

5088

In [32]:
nonfor_gain = aggregateByMuni("fad_changes_3.tif",
                               "towns.tif",
                               1)
nonfor_loss = aggregateByMuni("fad_changes_3.tif",
                               "towns.tif",
                               2)
nonfor_pp = aggregateByMuni("fad_changes_3.tif",
                             "towns.tif",
                             3)
nonfor_ap = aggregateByMuni("fad_changes_3.tif",
                            "towns.tif",
                            4)

In [33]:
print(nonfor_gain["val"].max())
print(nonfor_loss["val"].max())
print(nonfor_pp["val"].max())
print(nonfor_ap["val"].max())

4977
2595
122826
167009


### Calculate the Intensity
Output = df which contains the intensities for each municipality id.
- Loss Intensity = Loss/ Forest size at 2006
- Gain Intensity = Gain/ Forest size at 2016
- Change Intensity = Change Size (= Loss + Gain)/ Extent size at 2016

In [81]:
def calcChangeIntensityByMuni(gain_df,
                              loss_df,
                              size_df_2006,
                              size_df_2016):
    gain = gain_df.val.to_numpy()
    for_extent2016 = size_df_2016.fore.to_numpy()
    gain_intensity = (gain/ for_extent2016)* 100
    loss = loss_df.val.to_numpy()
    extent_size = size_df_2006.Extent.to_numpy()
    for_extent2006 = size_df_2006.fore.to_numpy()
    loss_intensity = (loss/ for_extent2006)* 100
    quantity = gain.astype(np.int64) - loss.astype(np.int64)
    quantity_abs = np.absolute(quantity)
    quantity_label = np.where(quantity == 0,
                              "zero",
                              np.where(quantity < 0,
                                       "loss",
                                       "gain"))
    gain_quantity = np.where(quantity_label == "gain",
                             quantity_abs,
                             0)
    loss_quantity = np.where(quantity_label == "loss",
                             quantity_abs,
                             0)
    category_exchange = np.where(quantity <= 0,
                                 (loss - quantity_abs)* 2,
                                 (gain - quantity_abs)* 2)
    exchange_gain_loss = category_exchange/ 2
    change = exchange_gain_loss + quantity_abs
    change_intensity = (change/ extent_size)* 100

    out_df = pd.DataFrame({
        "muni": gain_df.index,
        "fore_int16": size_df_2016.fore_intensity,
        "fore_int06": size_df_2006.fore_intensity,
        "gain": gain,
        "loss": loss,
        "gain_int": gain_intensity,
        "loss_int": loss_intensity,
        "extent": extent_size,
        "gain_quantity": gain_quantity,
        "loss_quantity": loss_quantity,
        "quant_lab": quantity_label,
        "catg_xchg": category_exchange,
        "xchg_gl": exchange_gain_loss,
        "change": change,
        "change_int": change_intensity
    })

    # out_df.fillna(0).to_csv("test.csv")
    return out_df.fillna(0)
    

In [82]:
frag_changes = calcChangeIntensityByMuni(frag_gain,
                                         frag_loss,
                                         frag2006,
                                         frag2016)

non_frag_changes = calcChangeIntensityByMuni(nonfrag_gain,
                                             nonfrag_loss,
                                             nonfrag2006,
                                             nonfrag2016)

non_forest_changes = calcChangeIntensityByMuni(nonfor_gain,
                                               nonfor_loss,
                                               nonforest2006,
                                               nonforest2016)


/var/folders/fj/5v0w95fn2cg7z3jtwpv6ldpw0000gn/T/ipykernel_2736/2369450989.py:7: RuntimeWarning: invalid value encountered in divide
  gain_intensity = (gain/ for_extent2016)* 100
/var/folders/fj/5v0w95fn2cg7z3jtwpv6ldpw0000gn/T/ipykernel_2736/2369450989.py:11: RuntimeWarning: invalid value encountered in divide
  loss_intensity = (loss/ for_extent2006)* 100


In [69]:
frag_changes

,muni,fore_int16,fore_int06,gain,loss,gain_int,loss_int,extent,gain_quantity,loss_quantity,quant_lab,catg_xchg,xchg_gl,change,change_int
muni,,,,,,,,,,,,,,,
2,2,36.363636,36.363636,0,0,0.000000,0.000000,11,0,0,zero,0.0,0.0,0.0,0.000000
8,8,0.000000,0.000000,0,0,0.000000,0.000000,3,0,0,zero,0.0,0.0,0.0,0.000000
9,9,0.000000,0.000000,0,0,0.000000,0.000000,23,0,0,zero,0.0,0.0,0.0,0.000000
10,10,0.000000,0.000000,0,0,0.000000,0.000000,40,0,0,zero,0.0,0.0,0.0,0.000000
11,11,0.000000,0.000000,0,0,0.000000,0.000000,7,0,0,zero,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,1232,0.000000,0.000000,0,0,0.000000,0.000000,4,0,0,zero,0.0,0.0,0.0,0.000000
1235,1235,0.000000,0.000000,0,0,0.000000,0.000000,8,0,0,zero,0.0,0.0,0.0,0.000000
1236,1236,13.652829,8.450819,3344,156,39.966535,3.012165,61284,3188,0,gain,312.0,156.0,3344.0,0.054566


In [70]:
non_frag_changes

,muni,fore_int16,fore_int06,gain,loss,gain_int,loss_int,extent,gain_quantity,loss_quantity,quant_lab,catg_xchg,xchg_gl,change,change_int
muni,,,,,,,,,,,,,,,
2,2,0.00000,0.000000,0,0,0.000000,0.000000,11,0,0,zero,0.0,0.0,0.0,0.000000
8,8,0.00000,0.000000,0,0,0.000000,0.000000,3,0,0,zero,0.0,0.0,0.0,0.000000
9,9,0.00000,0.000000,0,0,0.000000,0.000000,23,0,0,zero,0.0,0.0,0.0,0.000000
10,10,0.00000,0.000000,0,0,0.000000,0.000000,40,0,0,zero,0.0,0.0,0.0,0.000000
11,11,0.00000,0.000000,0,0,0.000000,0.000000,7,0,0,zero,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,1232,0.00000,0.000000,0,0,0.000000,0.000000,4,0,0,zero,0.0,0.0,0.0,0.000000
1235,1235,0.00000,0.000000,0,0,0.000000,0.000000,8,0,0,zero,0.0,0.0,0.0,0.000000
1236,1236,49.35546,55.453299,59,3796,0.195061,11.169962,61284,0,3737,loss,118.0,59.0,3796.0,0.061941


In [71]:
non_forest_changes

,muni,fore_int16,fore_int06,gain,loss,gain_int,loss_int,extent,gain_quantity,loss_quantity,quant_lab,catg_xchg,xchg_gl,change,change_int
muni,,,,,,,,,,,,,,,
2,2,63.636364,63.636364,0,0,0.000000,0.000000,11,0,0,zero,0.0,0.0,0.0,0.000000
8,8,100.000000,100.000000,0,0,0.000000,0.000000,3,0,0,zero,0.0,0.0,0.0,0.000000
9,9,100.000000,100.000000,0,0,0.000000,0.000000,23,0,0,zero,0.0,0.0,0.0,0.000000
10,10,100.000000,100.000000,0,0,0.000000,0.000000,40,0,0,zero,0.0,0.0,0.0,0.000000
11,11,100.000000,100.000000,0,0,0.000000,0.000000,7,0,0,zero,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,1232,100.000000,100.000000,0,0,0.000000,0.000000,4,0,0,zero,0.0,0.0,0.0,0.000000
1235,1235,100.000000,100.000000,0,0,0.000000,0.000000,8,0,0,zero,0.0,0.0,0.0,0.000000
1236,1236,36.991711,36.095881,673,124,2.968681,0.560553,61284,549,0,gain,248.0,124.0,673.0,0.010982


In [ ]:
# FORMAT DATA FRAME####


## Step4
### Join the results and the original vector file

In [83]:
v = v[["townID", "geometry"]]

In [84]:
frag_changes = frag_changes.drop("muni", axis = 1).reset_index()

gdf_frag = v.merge(frag_changes, how = "inner", left_on = "townID", right_on = "muni")
gdf_frag.to_file("./output/frag_changes.shp")

/var/folders/fj/5v0w95fn2cg7z3jtwpv6ldpw0000gn/T/ipykernel_2736/2433881801.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_frag.to_file("./output/frag_changes.shp")


In [85]:
non_frag_changes = non_frag_changes.drop("muni", axis = 1).reset_index()

gdf_nonfrag = v.merge(non_frag_changes, how = "inner", left_on = "townID", right_on = "muni")
gdf_nonfrag.to_file("./output/nonfrag_changes.shp")

/var/folders/fj/5v0w95fn2cg7z3jtwpv6ldpw0000gn/T/ipykernel_2736/2705534990.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nonfrag.to_file("./output/nonfrag_changes.shp")


In [86]:
non_forest_changes = non_forest_changes.drop("muni", axis = 1).reset_index()

gdf_nonfor = v.merge(non_forest_changes, how = "inner", left_on = "townID", right_on = "muni")
gdf_nonfor.to_file("./output/nonfor_changes.shp")

/var/folders/fj/5v0w95fn2cg7z3jtwpv6ldpw0000gn/T/ipykernel_2736/1296796206.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nonfor.to_file("./output/nonfor_changes.shp")


## Step 5

### Calculate Local Morans I

In [78]:
def calcMoran(gdf,
              variables_list,
              out_name):
    import pygeoda 

    file = pygeoda.open(gdf)
    w = pygeoda.queen_weights(file)

    d = {}
    for i in range(len(variables_list)):
        lm = pygeoda.local_moran(w, file[variables_list[i]])
        pval = pd.Series(lm.lisa_pvalues())
        lab = pd.Series(lm.lisa_clusters())
        p_key = f"p_{variables_list[i]}"
        l_key = f"l_{variables_list[i]}"
        d[p_key] = pval
        d[l_key] = lab

    d["geometry"] = gdf["geometry"]

    out_gdf = gpd.GeoDataFrame(d)
    out_gdf.to_file(out_name)


In [79]:
calcMoran(gdf_frag, ["gain_int", "loss_int", "change_int"], "./output/frag_int_moran.shp")

/var/folders/fj/5v0w95fn2cg7z3jtwpv6ldpw0000gn/T/ipykernel_2736/3951728541.py:22: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  out_gdf.to_file(out_name)


In [80]:
calcMoran(gdf_nonfrag, ["gain_int", "loss_int", "change_int"], "./output/nonfrag_int_moran.shp")
calcMoran(gdf_nonfor, ["gain_int", "loss_int", "change_int"], "./output/nonfor_int_moran.shp")

/var/folders/fj/5v0w95fn2cg7z3jtwpv6ldpw0000gn/T/ipykernel_2736/3951728541.py:22: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  out_gdf.to_file(out_name)
/var/folders/fj/5v0w95fn2cg7z3jtwpv6ldpw0000gn/T/ipykernel_2736/3951728541.py:22: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  out_gdf.to_file(out_name)


## Next Steps
1. Explain quantity and exchange (gross change vs net change)